In [1]:
import pandas as pd

# Load Data

In [2]:
data_dir = './'
raw_data = []
for year in range(2015, 2025):
    raw_data.append(pd.read_csv(data_dir + str(year) + '.csv'))

In [3]:
raw_data[9].tail()

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
253,E0,24/02/2024,15:00,Man United,Fulham,1,2,A,0,0,...,2.48,-0.75,1.75,2.05,1.83,2.11,1.90,2.14,1.80,2.06
254,E0,24/02/2024,17:30,Bournemouth,Man City,0,1,A,0,1,...,2.65,1.25,1.92,1.98,1.93,2.00,1.95,2.09,1.86,2.00
255,E0,24/02/2024,20:00,Arsenal,Newcastle,4,1,H,2,0,...,2.76,-1.50,2.01,1.89,2.02,1.91,2.04,1.94,1.97,1.90
256,E0,25/02/2024,13:30,Wolves,Sheffield United,1,0,H,1,0,...,2.10,-1.00,1.73,2.08,1.79,2.13,1.83,2.15,1.79,2.10
257,E0,26/02/2024,20:00,West Ham,Brentford,4,2,H,2,1,...,2.06,-0.25,2.01,1.89,2.03,1.90,2.06,1.90,2.02,1.85


# Select features

In [4]:
selected_columns = ['HomeTeam','AwayTeam','FTHG','FTAG','FTR']
seasons = []
for data in raw_data:
    seasons.append(data[selected_columns])

In [5]:
seasons[9].tail(10)

,HomeTeam,AwayTeam,FTHG,FTAG,FTR
248,Man City,Brentford,1,0,H
249,Liverpool,Luton,4,1,H
250,Aston Villa,Nott'm Forest,4,2,H
251,Brighton,Everton,1,1,D
252,Crystal Palace,Burnley,3,0,H
253,Man United,Fulham,1,2,A
254,Bournemouth,Man City,0,1,A
255,Arsenal,Newcastle,4,1,H
256,Wolves,Sheffield United,1,0,H
257,West Ham,Brentford,4,2,H


In [6]:
# 2024 use only 160 row because unbalaced data after 160 row
seasons[9] = seasons[9][:160]

In [7]:
for i in range(len(seasons)):
  print(f'{i}. {len(seasons[i])}')

for i in range(len(seasons)):
  seasons[i] = seasons[i].dropna()
  print(f'{i}. {len(seasons[i])}')

0. 381
1. 380
2. 380
3. 380
4. 380
5. 380
6. 380
7. 380
8. 380
9. 160
0. 380
1. 380
2. 380
3. 380
4. 380
5. 380
6. 380
7. 380
8. 380
9. 160


# Create new features

## AHTGS, AATGS, AHTGC, AATGC
* AHTGS - Aggreated Home Team Goal Scored ประตูรวมสะสมในฤดูกาลที่เจ้าบ้านทำได้ก่อนหน้า
* AATGS - Aggreated Away Team Goal Scored ประตูรวมสะสมในฤดูกาลที่ทีมเยือนทำได้ก่อนหน้า
* AHTGC - Aggreated Home Team Goal Conceded ประตูรวมสะสมในฤดูกาลที่เจ้าบ้านเสียให้คู่แข่งก่อนหน้า
* AATGC - Aggreated Away Team Goal Conceded ประตูรวมสะสมในฤดูกาลที่ทีมเยือนเสียให้คู่แข่งก่อนหน้า


### GS - Goal Scored

In [8]:
# Create DataFrame for aggregated goals scored by each teams till each matchweeks
def get_agg_goals_scored(season):
    # Create  a dictonary with team names as keys
    # { 'Team Name': [], ...}
    teams = {}
    for i in season['HomeTeam'].unique():  # get team name as index
      teams[i] = []

    # Goals scored each matchweek by each team (as either Home Team or Away Team)
    for i in range(len(season)):
        HTGS = season.iloc[i]['FTHG']
        ATGS = season.iloc[i]['FTAG']
        teams[season.iloc[i].HomeTeam].append(HTGS)
        teams[season.iloc[i].AwayTeam].append(ATGS)

    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    goals_scored = pd.DataFrame(data=teams, index=[i for i in range(1,(len(season) // 10) + 1)]).T
    goals_scored[0] = 0

    # Aggregate goals scored till each matchweek.
    for i in range(2,(len(season) // 10) + 1):
        goals_scored[i] = goals_scored[i] + goals_scored[i-1]

    return goals_scored

### GC - Goal Conceded

In [9]:
# Create DataFrame for aggregated goals conceded by each teams till each matchweeks
def get_agg_goals_conceded(season):
    # Create  a dictonary with team names as key
    teams = {}
    for i in season['HomeTeam'].unique():  # get team name as index
        teams[i] = []

    # Goals conceded each matchweek by each team (as either Home Team or Away Team)
    for i in range(len(season)):
        ATGC = season.iloc[i]['FTHG']
        HTGC = season.iloc[i]['FTAG']
        teams[season.iloc[i].HomeTeam].append(HTGC)
        teams[season.iloc[i].AwayTeam].append(ATGC)

    # Create a dataframe for goals conceded where rows are teams and cols are matchweek.
    goals_conceded = pd.DataFrame(data=teams, index=[i for i in range(1,(len(season) // 10) + 1)]).T
    goals_conceded[0] = 0

    # Aggregate goals conceded till each matchweek.
    for i in range(2,(len(season) // 10) + 1):
        goals_conceded[i] = goals_conceded[i] + goals_conceded[i-1]

    return goals_conceded

### Split GS and GC to Home Team and Away Team

In [10]:
def get_gss(season):
    AGS = get_agg_goals_scored(season)
    AGC = get_agg_goals_conceded(season)

    j = 0
    AHTGS = []
    AATGS = []
    AHTGC = []
    AATGC = []

    for i in range(len(season)):
        ht = season.iloc[i].HomeTeam
        at = season.iloc[i].AwayTeam
        AHTGS.append(AGS.loc[ht][j])
        AATGS.append(AGS.loc[at][j])
        AHTGC.append(AGC.loc[ht][j])
        AATGC.append(AGC.loc[at][j])

        if ((i + 1) % 10) == 0:
            j = j + 1

    season['AHTGS'] = AHTGS
    season['AATGS'] = AATGS
    season['AHTGC'] = AHTGC
    season['AATGC'] = AATGC

    return season

Apply to all dataset

In [11]:
# Apply to each season
for i in range(len(seasons)):
    seasons[i] = get_gss(seasons[i])

In [12]:
seasons[9]

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,AHTGS,AATGS,AHTGC,AATGC
0,Burnley,Man City,0,3,A,0,0,0,0
1,Arsenal,Nott'm Forest,2,1,H,0,0,0,0
2,Bournemouth,West Ham,1,1,D,0,0,0,0
3,Brighton,Luton,4,1,H,0,0,0,0
4,Everton,Fulham,0,1,A,0,0,0,0
...,...,...,...,...,...,...,...,...,...
155,Aston Villa,Arsenal,1,0,H,34,33,20,14
156,Everton,Chelsea,2,0,H,18,26,20,24
157,Fulham,West Ham,5,0,H,21,26,26,25
158,Luton,Man City,1,2,A,16,36,30,17


## HTP, ATP

RESPECTIVE POINTS (HTP, ATP) \

 * HTP - คะแนนรวมที่เจ้าบ้านได้ \
 * ATP - คะแนนรวมที่ทีมเยือนได้ \
จะคำนวนจากผล FTR ทีมที่ชนะจะได้ 3 คะแนน เสมอได้ 1 คะแนน แพ้ 0 คะแนน



In [13]:
# get result of each team in each match
def get_match_results(season):
    # Create dictionary with team names as keys
    teams = {}
    for i in season['HomeTeam'].unique():
        teams[i] = []

    # the value corresponding to keys is a list containing the match result
    for i in range(len(season)):
        if season.iloc[i].FTR == 'H':
            teams[season.iloc[i].HomeTeam].append('W')
            teams[season.iloc[i].AwayTeam].append('L')
        elif season.iloc[i].FTR == 'A':
            teams[season.iloc[i].HomeTeam].append('L')
            teams[season.iloc[i].AwayTeam].append('W')
        else:
            teams[season.iloc[i].HomeTeam].append('D')
            teams[season.iloc[i].AwayTeam].append('D')

    return pd.DataFrame(data=teams, index=[i for i in range(1, (len(season) // 10) + 1)]).T

In [14]:
# test get_match_results function
get_match_results(seasons[9])

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
Burnley,L,L,L,D,L,L,W,L,L,L,L,L,L,W,L,D
Arsenal,W,W,D,W,W,D,W,W,D,W,L,W,W,W,W,L
Bournemouth,D,L,L,D,D,L,L,L,L,W,L,W,W,D,W,W
Brighton,W,W,L,W,W,W,L,D,L,D,D,D,W,L,W,D
Everton,L,L,L,D,L,W,L,W,L,W,D,W,L,W,W,W
Sheffield United,L,L,L,D,L,L,L,L,L,L,W,D,L,L,L,W
Newcastle,W,L,L,L,W,W,W,D,W,D,W,L,W,W,L,L
Brentford,D,W,D,D,L,L,D,L,W,W,W,L,L,W,L,L
Chelsea,D,L,W,L,D,L,W,W,D,L,W,D,L,W,L,L
Man United,W,L,W,L,L,W,L,W,W,L,W,W,W,L,W,L


In [15]:
# Apply W=3 , D=1 , L=0
def get_points(result):
    if result == 'W':
        return 3
    elif result == 'D':
        return 1
    else:
        return 0

# Apply to cumulative form
def get_cuml_points(match_results, season):
    matchres_points = match_results.applymap(get_points)
    for i in range(2, (len(season) // 10) + 1):
        matchres_points[i] = matchres_points[i] + matchres_points[i-1]

    matchres_points.insert(column=0, loc=0, value=[0 * i for i in range(20)])
    return matchres_points

In [16]:
# test get_cuml_points function
get_cuml_points(get_match_results(seasons[9]),seasons[9])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
Burnley,0,0,0,0,1,1,1,4,4,4,4,4,4,4,7,7,8
Arsenal,0,3,6,7,10,13,14,17,20,21,24,24,27,30,33,36,36
Bournemouth,0,1,1,1,2,3,3,3,3,3,6,6,9,12,13,16,19
Brighton,0,3,6,6,9,12,15,15,16,16,17,18,19,22,22,25,26
Everton,0,0,0,0,1,1,4,4,7,7,10,11,14,14,17,20,23
Sheffield United,0,0,0,0,1,1,1,1,1,1,1,4,5,5,5,5,8
Newcastle,0,3,3,3,3,6,9,12,13,16,17,20,20,23,26,26,26
Brentford,0,1,4,5,6,6,6,7,7,10,13,16,16,16,19,19,19
Chelsea,0,1,1,4,4,5,5,8,11,12,12,15,16,16,19,19,19
Man United,0,3,3,6,6,6,9,9,12,15,15,18,21,24,24,27,27


In [17]:
# function that get HTP and ATP features
# HTP - Home Team Points
# ATP - Away Team Points

def get_agg_points(season):
    match_results = get_match_results(season)
    cum_pts = get_cuml_points(match_results, season)
    HTP = []
    ATP = []
    j = 0
    for i in range(len(season)):
        ht = season.iloc[i].HomeTeam
        at = season.iloc[i].AwayTeam
        HTP.append(cum_pts.loc[ht][j])
        ATP.append(cum_pts.loc[at][j])

        if ((i + 1) % 10) == 0:
            j = j + 1

    season.loc[:,'HTP'] = HTP
    season.loc[:,'ATP'] = ATP

    return season

In [18]:
# Apply to each season
for i in range(len(seasons)):
    seasons[i] = get_agg_points(seasons[i])

In [19]:
seasons[9].tail(20)

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,AHTGS,AATGS,AHTGC,AATGC,HTP,ATP
140,Wolves,Burnley,1,0,H,19,15,25,32,15,7
141,Luton,Arsenal,3,4,A,13,29,26,11,9,33
142,Brighton,Brentford,2,1,H,30,22,26,19,22,19
143,Crystal Palace,Bournemouth,0,2,A,14,16,19,30,16,13
144,Fulham,Nott'm Forest,5,0,H,16,16,26,22,15,13
145,Sheffield United,Liverpool,0,2,A,11,32,39,14,5,31
146,Aston Villa,Man City,1,0,H,33,36,20,16,29,30
147,Man United,Chelsea,2,1,H,16,25,17,22,24,19
148,Everton,Newcastle,3,0,H,15,32,20,14,17,26
149,Tottenham,West Ham,1,2,A,28,24,20,24,27,21


## HM1, HM2, HM3, AM1, AM2, AM3

GET TEAM FORM (HM1, HM2, HM3, AM1, AM2, AM3) \
 * HM1, HM2, HM3 - ผลการแข็งขันของทีมเจ้าบ้านก่อนหน้า 1, 2, 3 นัด \
 * AM1, AM2, AM3 - ผลการแข็งขันของทีมเก่อนหน้า 1, 2, 3 นัด

In [20]:
# get form from 3 result before for each team
def get_form(season, num):          # the num th before
    form = get_match_results(season)
    form_final = form.copy()
    for i in range(num, (len(season) // 10) + 1):
        form_final[i] = ''
        j = 0
        while j < num:
            form_final[i] += form[i-j]
            j += 1
    return form_final

In [21]:
# test get_form with last 2 match result of each team
get_form(seasons[9],2)

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
Burnley,L,LL,LL,DL,LD,LL,WL,LW,LL,LL,LL,LL,LL,WL,LW,DL
Arsenal,W,WW,DW,WD,WW,DW,WD,WW,DW,WD,LW,WL,WW,WW,WW,LW
Bournemouth,D,LD,LL,DL,DD,LD,LL,LL,LL,WL,LW,WL,WW,DW,WD,WW
Brighton,W,WW,LW,WL,WW,WW,LW,DL,LD,DL,DD,DD,WD,LW,WL,DW
Everton,L,LL,LL,DL,LD,WL,LW,WL,LW,WL,DW,WD,LW,WL,WW,WW
Sheffield United,L,LL,LL,DL,LD,LL,LL,LL,LL,LL,WL,DW,LD,LL,LL,WL
Newcastle,W,LW,LL,LL,WL,WW,WW,DW,WD,DW,WD,LW,WL,WW,LW,LL
Brentford,D,WD,DW,DD,LD,LL,DL,LD,WL,WW,WW,LW,LL,WL,LW,LL
Chelsea,D,LD,WL,LW,DL,LD,WL,WW,DW,LD,WL,DW,LD,WL,LW,LL
Man United,W,LW,WL,LW,LL,WL,LW,WL,WW,LW,WL,WW,WW,LW,WL,LW


In [22]:
# get HM and AM features
def add_form(season, num):
    form = get_form(season, num)
    h = ['M' for i in range(num * 10)]    # since form is not available for n MW (n*10)
    a = ['M' for i in range(num * 10)]

    j = num
    for i in range((num * 10), len(season)):
        ht = season.iloc[i].HomeTeam
        at = season.iloc[i].AwayTeam

        past = form.loc[ht][j]    # get past n results
        h.append(past[num - 1])   # 0 index is most recent

        past = form.loc[at][j]    # get past n results
        a.append(past[num - 1])   # 0 in dex is most recent

        if ((i + 1) % 10) == 0:
            j = j + 1

    season['HM' + str(num)] = h
    season['AM' + str(num)] = a

    return season

In [23]:
# test add_form with last 2 match result of each team HM,AM features
add_form(seasons[9], 2)

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,AHTGS,AATGS,AHTGC,AATGC,HTP,ATP,HM2,AM2
0,Burnley,Man City,0,3,A,0,0,0,0,0,0,M,M
1,Arsenal,Nott'm Forest,2,1,H,0,0,0,0,0,0,M,M
2,Bournemouth,West Ham,1,1,D,0,0,0,0,0,0,M,M
3,Brighton,Luton,4,1,H,0,0,0,0,0,0,M,M
4,Everton,Fulham,0,1,A,0,0,0,0,0,0,M,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,Aston Villa,Arsenal,1,0,H,34,33,20,14,32,36,D,W
156,Everton,Chelsea,2,0,H,18,26,20,24,20,19,W,W
157,Fulham,West Ham,5,0,H,21,26,26,25,18,24,L,D
158,Luton,Man City,1,2,A,16,36,30,17,9,30,L,D


In [24]:
# get HM,AM with last 1,2,3 match result in a row
def add_form_df(season):
    season = add_form(season, 1)
    season = add_form(season, 2)
    season = add_form(season, 3)
    return season

In [25]:
# Apply to each season
for i in range(len(seasons)):
    seasons[i] = add_form_df(seasons[i])

In [26]:
seasons[9]

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,AHTGS,AATGS,AHTGC,AATGC,HTP,ATP,HM2,AM2,HM1,AM1,HM3,AM3
0,Burnley,Man City,0,3,A,0,0,0,0,0,0,M,M,M,M,M,M
1,Arsenal,Nott'm Forest,2,1,H,0,0,0,0,0,0,M,M,M,M,M,M
2,Bournemouth,West Ham,1,1,D,0,0,0,0,0,0,M,M,M,M,M,M
3,Brighton,Luton,4,1,H,0,0,0,0,0,0,M,M,M,M,M,M
4,Everton,Fulham,0,1,A,0,0,0,0,0,0,M,M,M,M,M,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,Aston Villa,Arsenal,1,0,H,34,33,20,14,32,36,D,W,W,W,W,W
156,Everton,Chelsea,2,0,H,18,26,20,24,20,19,W,W,W,L,L,L
157,Fulham,West Ham,5,0,H,21,26,26,25,18,24,L,D,W,W,W,W
158,Luton,Man City,1,2,A,16,36,30,17,9,30,L,D,L,L,W,D


## League Points
performance metric between a home team and an away team
 * HomeTeamLP - อันดับทีมเจ้าบ้านเมื่อฤดูกาลก่อน
 * AwayTeamLP - อันดับทีมเยือนเมื่อฤดูกาลก่อน

In [27]:
standings = pd.read_csv('EPLStandings.csv')
standings.set_index(['Team'], inplace=True)
standings = standings.fillna(20)
standings

,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
Team,,,,,,,,,,
Arsenal,3.0,2.0,5.0,6.0,5.0,8.0,8.0,5.0,2.0,2.0
Aston Villa,17.0,20.0,20.0,20.0,20.0,17.0,11.0,14.0,7.0,3.0
Birmingham,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
Blackburn,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
Blackpool,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
Bolton,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0
Bournemouth,20.0,16.0,9.0,12.0,14.0,18.0,20.0,20.0,15.0,14.0
Brentford,20.0,20.0,20.0,20.0,20.0,20.0,20.0,13.0,9.0,11.0
Brighton,20.0,20.0,20.0,15.0,17.0,15.0,16.0,9.0,6.0,8.0


### HomeTeamLP, AwayTeamLP features

In [28]:
# get HomeTeamLP, AwayTeamLP features
def get_last(season, standings, year):
    home_team_lp = []
    away_team_lp = []
    for i in range(len(season)):
        ht = season.iloc[i].HomeTeam
        at = season.iloc[i].AwayTeam
        home_team_lp.append(standings.loc[ht][year])
        away_team_lp.append(standings.loc[at][year])
    season['HomeTeamLP'] = home_team_lp
    season['AwayTeamLP'] = away_team_lp
    return season

In [29]:
# Apply to each season
for i in range(len(seasons)):
    seasons[i] = get_last(seasons[i], standings, i)

In [30]:
seasons[9]

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,AHTGS,AATGS,AHTGC,AATGC,HTP,ATP,HM2,AM2,HM1,AM1,HM3,AM3,HomeTeamLP,AwayTeamLP
0,Burnley,Man City,0,3,A,0,0,0,0,0,0,M,M,M,M,M,M,19.0,4.0
1,Arsenal,Nott'm Forest,2,1,H,0,0,0,0,0,0,M,M,M,M,M,M,2.0,17.0
2,Bournemouth,West Ham,1,1,D,0,0,0,0,0,0,M,M,M,M,M,M,14.0,9.0
3,Brighton,Luton,4,1,H,0,0,0,0,0,0,M,M,M,M,M,M,8.0,18.0
4,Everton,Fulham,0,1,A,0,0,0,0,0,0,M,M,M,M,M,M,15.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,Aston Villa,Arsenal,1,0,H,34,33,20,14,32,36,D,W,W,W,W,W,3.0,2.0
156,Everton,Chelsea,2,0,H,18,26,20,24,20,19,W,W,W,L,L,L,15.0,12.0
157,Fulham,West Ham,5,0,H,21,26,26,25,18,24,L,D,W,W,W,W,10.0,9.0
158,Luton,Man City,1,2,A,16,36,30,17,9,30,L,D,L,L,W,D,18.0,4.0


### MW features
MW = Match week ( 1 MW = 10 คู่ ใน 1 สัปดาห์)

In [31]:
def get_mw(season):
    j = 1
    match_week = []
    for i in range(len(season)):
        match_week.append(j)
        if ((i + 1) % 10) == 0:
            j = j + 1
    season['MW'] = match_week
    return season

# Apply to each season
for i in range(len(seasons)):
    seasons[i] = get_mw(seasons[i])

In [32]:
seasons[9]

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,AHTGS,AATGS,AHTGC,AATGC,HTP,ATP,HM2,AM2,HM1,AM1,HM3,AM3,HomeTeamLP,AwayTeamLP,MW
0,Burnley,Man City,0,3,A,0,0,0,0,0,0,M,M,M,M,M,M,19.0,4.0,1
1,Arsenal,Nott'm Forest,2,1,H,0,0,0,0,0,0,M,M,M,M,M,M,2.0,17.0,1
2,Bournemouth,West Ham,1,1,D,0,0,0,0,0,0,M,M,M,M,M,M,14.0,9.0,1
3,Brighton,Luton,4,1,H,0,0,0,0,0,0,M,M,M,M,M,M,8.0,18.0,1
4,Everton,Fulham,0,1,A,0,0,0,0,0,0,M,M,M,M,M,M,15.0,10.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,Aston Villa,Arsenal,1,0,H,34,33,20,14,32,36,D,W,W,W,W,W,3.0,2.0,16
156,Everton,Chelsea,2,0,H,18,26,20,24,20,19,W,W,W,L,L,L,15.0,12.0,16
157,Fulham,West Ham,5,0,H,21,26,26,25,18,24,L,D,W,W,W,W,10.0,9.0,16
158,Luton,Man City,1,2,A,16,36,30,17,9,30,L,D,L,L,W,D,18.0,4.0,16


### concatenation of all seasons

In [33]:
gameplays = pd.concat(seasons, ignore_index=True)
gameplays

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,AHTGS,AATGS,AHTGC,AATGC,HTP,ATP,HM1,AM1,HM2,AM2,HM3,AM3,HomeTeamLP,AwayTeamLP,MW
0,Arsenal,Crystal Palace,2.0,1.0,H,0.0,0.0,0.0,0.0,0,0,M,M,M,M,M,M,3.0,10.0,1
1,Leicester,Everton,2.0,2.0,D,0.0,0.0,0.0,0.0,0,0,M,M,M,M,M,M,14.0,11.0,1
2,Man United,Swansea,1.0,2.0,A,0.0,0.0,0.0,0.0,0,0,M,M,M,M,M,M,4.0,8.0,1
3,QPR,Hull,0.0,1.0,A,0.0,0.0,0.0,0.0,0,0,M,M,M,M,M,M,20.0,18.0,1
4,Stoke,Aston Villa,0.0,1.0,A,0.0,0.0,0.0,0.0,0,0,M,M,M,M,M,M,9.0,17.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3575,Aston Villa,Arsenal,1.0,0.0,H,34.0,33.0,20.0,14.0,32,36,W,W,D,W,W,W,3.0,2.0,16
3576,Everton,Chelsea,2.0,0.0,H,18.0,26.0,20.0,24.0,20,19,W,L,W,W,L,L,15.0,12.0,16
3577,Fulham,West Ham,5.0,0.0,H,21.0,26.0,26.0,25.0,18,24,W,W,L,D,W,W,10.0,9.0,16
3578,Luton,Man City,1.0,2.0,A,16.0,36.0,30.0,17.0,9,30,L,L,L,D,W,D,18.0,4.0,16


### DiffLP feature
Difference in League Points \
DiffLP = HomeTeamLP - AwayTeamLP

In [34]:
# Diff in last year positions
gameplays['DiffLP'] = gameplays['HomeTeamLP'] - gameplays['AwayTeamLP']

In [35]:
gameplays

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,AHTGS,AATGS,AHTGC,AATGC,HTP,...,HM1,AM1,HM2,AM2,HM3,AM3,HomeTeamLP,AwayTeamLP,MW,DiffLP
0,Arsenal,Crystal Palace,2.0,1.0,H,0.0,0.0,0.0,0.0,0,...,M,M,M,M,M,M,3.0,10.0,1,-7.0
1,Leicester,Everton,2.0,2.0,D,0.0,0.0,0.0,0.0,0,...,M,M,M,M,M,M,14.0,11.0,1,3.0
2,Man United,Swansea,1.0,2.0,A,0.0,0.0,0.0,0.0,0,...,M,M,M,M,M,M,4.0,8.0,1,-4.0
3,QPR,Hull,0.0,1.0,A,0.0,0.0,0.0,0.0,0,...,M,M,M,M,M,M,20.0,18.0,1,2.0
4,Stoke,Aston Villa,0.0,1.0,A,0.0,0.0,0.0,0.0,0,...,M,M,M,M,M,M,9.0,17.0,1,-8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3575,Aston Villa,Arsenal,1.0,0.0,H,34.0,33.0,20.0,14.0,32,...,W,W,D,W,W,W,3.0,2.0,16,1.0
3576,Everton,Chelsea,2.0,0.0,H,18.0,26.0,20.0,24.0,20,...,W,L,W,W,L,L,15.0,12.0,16,3.0
3577,Fulham,West Ham,5.0,0.0,H,21.0,26.0,26.0,25.0,18,...,W,W,L,D,W,W,10.0,9.0,16,1.0
3578,Luton,Man City,1.0,2.0,A,16.0,36.0,30.0,17.0,9,...,L,L,L,D,W,D,18.0,4.0,16,14.0


## HTGD, ATGD features
* HTGD - ส่วนต่างประตูได้เสียของทีมเจ้าบ้าน
* ATGD - ส่วนต่างประตูได้เสียของทีมเยือน

In [36]:
# Get Goal Difference
gameplays['HTGD'] = gameplays['AHTGS'] - gameplays['AHTGC']
gameplays['ATGD'] = gameplays['AATGS'] - gameplays['AATGC']

In [37]:
gameplays

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,AHTGS,AATGS,AHTGC,AATGC,HTP,...,HM2,AM2,HM3,AM3,HomeTeamLP,AwayTeamLP,MW,DiffLP,HTGD,ATGD
0,Arsenal,Crystal Palace,2.0,1.0,H,0.0,0.0,0.0,0.0,0,...,M,M,M,M,3.0,10.0,1,-7.0,0.0,0.0
1,Leicester,Everton,2.0,2.0,D,0.0,0.0,0.0,0.0,0,...,M,M,M,M,14.0,11.0,1,3.0,0.0,0.0
2,Man United,Swansea,1.0,2.0,A,0.0,0.0,0.0,0.0,0,...,M,M,M,M,4.0,8.0,1,-4.0,0.0,0.0
3,QPR,Hull,0.0,1.0,A,0.0,0.0,0.0,0.0,0,...,M,M,M,M,20.0,18.0,1,2.0,0.0,0.0
4,Stoke,Aston Villa,0.0,1.0,A,0.0,0.0,0.0,0.0,0,...,M,M,M,M,9.0,17.0,1,-8.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3575,Aston Villa,Arsenal,1.0,0.0,H,34.0,33.0,20.0,14.0,32,...,D,W,W,W,3.0,2.0,16,1.0,14.0,19.0
3576,Everton,Chelsea,2.0,0.0,H,18.0,26.0,20.0,24.0,20,...,W,W,L,L,15.0,12.0,16,3.0,-2.0,2.0
3577,Fulham,West Ham,5.0,0.0,H,21.0,26.0,26.0,25.0,18,...,L,D,W,W,10.0,9.0,16,1.0,-5.0,1.0
3578,Luton,Man City,1.0,2.0,A,16.0,36.0,30.0,17.0,9,...,L,D,W,D,18.0,4.0,16,14.0,-14.0,19.0


## DiffFormPts feature


* HTFormPtsStr = HM1 + HM2 + HM3 string
* ATFormPtsStr = AM1 + AM2 + AM3 string

* HTFormPts = sum of HTFormPtsStr that 'W'=3, 'D'=1, 'L'=0
* ATFormPts = sum of ATFormPtsStr that 'W'=3, 'D'=1, 'L'=0

* DiffFormPts = Points difference (HTP - ATP) calculate from last 1,2,3 result points of each team \
DiffFormPts = HTFormPts - ATFormPts

In [38]:
# Gets the form points.
def get_form_points(string):
    sum = 0
    for letter in string:
        sum += get_points(letter)
    return sum

gameplays['HTFormPtsStr'] = gameplays['HM1'] + gameplays['HM2'] + gameplays['HM3']
gameplays['ATFormPtsStr'] = gameplays['AM1'] + gameplays['AM2'] + gameplays['AM3']

gameplays['HTFormPts'] = gameplays['HTFormPtsStr'].apply(get_form_points)
gameplays['ATFormPts'] = gameplays['ATFormPtsStr'].apply(get_form_points)

# Get difference form points
gameplays['DiffFormPts'] = gameplays['HTFormPts'] - gameplays['ATFormPts']

In [39]:
gameplays

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,AHTGS,AATGS,AHTGC,AATGC,HTP,...,AwayTeamLP,MW,DiffLP,HTGD,ATGD,HTFormPtsStr,ATFormPtsStr,HTFormPts,ATFormPts,DiffFormPts
0,Arsenal,Crystal Palace,2.0,1.0,H,0.0,0.0,0.0,0.0,0,...,10.0,1,-7.0,0.0,0.0,MMM,MMM,0,0,0
1,Leicester,Everton,2.0,2.0,D,0.0,0.0,0.0,0.0,0,...,11.0,1,3.0,0.0,0.0,MMM,MMM,0,0,0
2,Man United,Swansea,1.0,2.0,A,0.0,0.0,0.0,0.0,0,...,8.0,1,-4.0,0.0,0.0,MMM,MMM,0,0,0
3,QPR,Hull,0.0,1.0,A,0.0,0.0,0.0,0.0,0,...,18.0,1,2.0,0.0,0.0,MMM,MMM,0,0,0
4,Stoke,Aston Villa,0.0,1.0,A,0.0,0.0,0.0,0.0,0,...,17.0,1,-8.0,0.0,0.0,MMM,MMM,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3575,Aston Villa,Arsenal,1.0,0.0,H,34.0,33.0,20.0,14.0,32,...,2.0,16,1.0,14.0,19.0,WDW,WWW,7,9,-2
3576,Everton,Chelsea,2.0,0.0,H,18.0,26.0,20.0,24.0,20,...,12.0,16,3.0,-2.0,2.0,WWL,LWL,6,3,3
3577,Fulham,West Ham,5.0,0.0,H,21.0,26.0,26.0,25.0,18,...,9.0,16,1.0,-5.0,1.0,WLW,WDW,6,7,-1
3578,Luton,Man City,1.0,2.0,A,16.0,36.0,30.0,17.0,9,...,4.0,16,14.0,-14.0,19.0,LLW,LDD,3,2,1


## Scale HTP, ATP, HTGD, ATGD, DiffFormPts.
Scale the HTP, ATP, HTGD, ATGD, DiffFormPts values with MW or average the values per game

In [40]:
# Scale HTP, ATP, HTGD, ATGD, DiffFormPts.
cols = ['HTP','ATP','HTGD','ATGD','DiffFormPts']
gameplays.MW = gameplays.MW.astype(float)

for col in cols:
    gameplays[col] = gameplays[col] / gameplays.MW

In [41]:
gameplays

,HomeTeam,AwayTeam,FTHG,FTAG,FTR,AHTGS,AATGS,AHTGC,AATGC,HTP,...,AwayTeamLP,MW,DiffLP,HTGD,ATGD,HTFormPtsStr,ATFormPtsStr,HTFormPts,ATFormPts,DiffFormPts
0,Arsenal,Crystal Palace,2.0,1.0,H,0.0,0.0,0.0,0.0,0.0000,...,10.0,1.0,-7.0,0.0000,0.0000,MMM,MMM,0,0,0.0000
1,Leicester,Everton,2.0,2.0,D,0.0,0.0,0.0,0.0,0.0000,...,11.0,1.0,3.0,0.0000,0.0000,MMM,MMM,0,0,0.0000
2,Man United,Swansea,1.0,2.0,A,0.0,0.0,0.0,0.0,0.0000,...,8.0,1.0,-4.0,0.0000,0.0000,MMM,MMM,0,0,0.0000
3,QPR,Hull,0.0,1.0,A,0.0,0.0,0.0,0.0,0.0000,...,18.0,1.0,2.0,0.0000,0.0000,MMM,MMM,0,0,0.0000
4,Stoke,Aston Villa,0.0,1.0,A,0.0,0.0,0.0,0.0,0.0000,...,17.0,1.0,-8.0,0.0000,0.0000,MMM,MMM,0,0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3575,Aston Villa,Arsenal,1.0,0.0,H,34.0,33.0,20.0,14.0,2.0000,...,2.0,16.0,1.0,0.8750,1.1875,WDW,WWW,7,9,-0.1250
3576,Everton,Chelsea,2.0,0.0,H,18.0,26.0,20.0,24.0,1.2500,...,12.0,16.0,3.0,-0.1250,0.1250,WWL,LWL,6,3,0.1875
3577,Fulham,West Ham,5.0,0.0,H,21.0,26.0,26.0,25.0,1.1250,...,9.0,16.0,1.0,-0.3125,0.0625,WLW,WDW,6,7,-0.0625
3578,Luton,Man City,1.0,2.0,A,16.0,36.0,30.0,17.0,0.5625,...,4.0,16.0,14.0,-0.8750,1.1875,LLW,LDD,3,2,0.0625


In [42]:
gameplays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3580 entries, 0 to 3579
Data columns (total 28 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomeTeam      3580 non-null   object 
 1   AwayTeam      3580 non-null   object 
 2   FTHG          3580 non-null   float64
 3   FTAG          3580 non-null   float64
 4   FTR           3580 non-null   object 
 5   AHTGS         3580 non-null   float64
 6   AATGS         3580 non-null   float64
 7   AHTGC         3580 non-null   float64
 8   AATGC         3580 non-null   float64
 9   HTP           3580 non-null   float64
 10  ATP           3580 non-null   float64
 11  HM1           3580 non-null   object 
 12  AM1           3580 non-null   object 
 13  HM2           3580 non-null   object 
 14  AM2           3580 non-null   object 
 15  HM3           3580 non-null   object 
 16  AM3           3580 non-null   object 
 17  HomeTeamLP    3580 non-null   float64
 18  AwayTeamLP    3580 non-null 

In [44]:
gameplays.describe()

,FTHG,FTAG,AHTGS,AATGS,AHTGC,AATGC,HTP,ATP,HomeTeamLP,AwayTeamLP,MW,DiffLP,HTGD,ATGD,HTFormPts,ATFormPts,DiffFormPts
count,3580.000000,3580.000000,3580.000000,3580.000000,3580.000000,3580.000000,3580.000000,3580.000000,3580.000000,3580.000000,3580.000000,3580.000000,3580.000000,3580.000000,3580.000000,3580.000000,3580.000000
mean,1.529888,1.226257,24.674581,24.746089,24.732402,24.613128,1.213906,1.222726,10.505028,10.494972,19.008380,0.010056,-0.006713,0.007374,3.844693,3.992179,-0.008049
std,1.316370,1.192405,18.061658,17.979920,16.668200,16.648556,0.566920,0.567909,5.766116,5.768054,11.001152,8.370935,0.778467,0.774613,2.548312,2.592453,0.356726
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,-19.000000,-2.800000,-2.500000,0.000000,0.000000,-2.000000
25%,1.000000,0.000000,10.000000,11.000000,11.000000,11.000000,0.863636,0.875000,6.000000,5.000000,9.000000,-6.000000,-0.515396,-0.500000,2.000000,2.000000,-0.142857
50%,1.000000,1.000000,22.000000,22.000000,23.000000,23.000000,1.185185,1.194444,11.000000,10.000000,19.000000,-1.000000,-0.097619,-0.076923,4.000000,4.000000,0.000000
75%,2.000000,2.000000,36.000000,36.000000,36.000000,36.250000,1.593750,1.600000,15.250000,15.250000,29.000000,6.000000,0.473684,0.500000,6.000000,6.000000,0.114560
max,9.000000,9.000000,102.000000,105.000000,79.000000,78.000000,2.821429,2.821429,20.000000,20.000000,38.000000,19.000000,2.777778,2.800000,9.000000,9.000000,1.666667


# Save dataset

In [43]:
# save file
gameplays.to_csv('../final_dataset.csv')